In [89]:
import numpy as np

import random
import math
import time
import tqdm
import matplotlib.pyplot as plt
from scipy import sparse
import seaborn as sns

import os
os.environ["JAX_PLATFORM_NAME"] = "cpu"

PATH = os.getcwd()

### Generate Adjacency Matrices

In [90]:
class Node:
    def __init__(self, position, xadj, yadj):
        self.position = position
        self.xadj = xadj
        self.yadj = yadj
    
    def __str__(self):
        return f"{self.position} {self.xadj} {self.yadj}"
    
def squareAdjacencyList(a,b): # constructs a periodic adjacency graph with width a and height b
    nodes = []
    for j in range(0,b):
        for i in range(0,a):
            xadj = [[(i-1) % a,j],[(i+1) % a,j]]
            yadj = [[i,(j-1) % b],[i,(j+1) % b]]
            nodes.append(Node([[i,j]],xadj,yadj))

    return nodes

def firstneighbors(a,b): 
    nodes = squareAdjacencyList(a,b)
    N = a*b
    J = [[0 for col in range(N)] for row in range(N)]

    for i in range(0,N-1):
        for j in range(i+1,N):
            flag = False
            for xptr in nodes[i].xadj:
                if xptr in nodes[j].position:
                    flag = True
            for yptr in nodes[i].yadj:
                if yptr in nodes[j].position:
                    flag = True
            if flag:
                J[i][j] = 1
                J[j][i] = 1
        
    return np.array(J)

def secondneighbors(a,b):
    nodes = squareAdjacencyList(a,b)
    N = a*b
    J = [[0 for col in range(N)] for row in range(N)]

    for i in range(0,N-1):
        for j in range(i+1,N):
            flag = False

            for xptr in nodes[i].xadj:
                try:
                    for k in range(0,N):
                        if xptr in nodes[k].position:
                            intermediate = k
                    for yptr in nodes[intermediate].yadj:
                        if yptr in nodes[j].position:
                            flag = True
                except UnboundLocalError:
                    pass

            if flag:
                J[i][j] = 1
                J[j][i] = 1

    return np.array(J)

### Define State Class

In [91]:
class State:
    def __init__(self, a, b, spinors):
        self.a = a
        self.b = b
        self.spinors = spinors # n-dimensional array of ordered pairs representing the spin state of each site
        self.adjacencies(a,b)
        self.numSites(a,b)   
    def numSites(self,a,b):
        self.n = a*b    
    def adjacencies(self,a,b):
        self.N1 = firstneighbors(a,b)
        self.N2 = secondneighbors(a,b)
        
    def energy(self,J1,J2):
        n = self.n
        spinors = self.spinors
        N1 = self.N1
        N2 = self.N2
        
        sum = 0.0
        for i in range(n-1):
            for j in range(i+1,n):
                if N1[i][j] != 0 or N2[i][j] != 0:
                    spin1 = spinors[i]/np.linalg.norm(spinors[i])
                    spin2 = spinors[j]/np.linalg.norm(spinors[j])
                    zCoupling = spin1[0]*spin2[0]+spin1[1]*spin2[1]-spin1[0]*spin2[1]-spin1[1]*spin2[0]
                    xyCoupling = 2*(spin1[0]*spin2[1]+spin1[1]*spin2[0])
                if N1[i][j] != 0:
                    sum += J1*(zCoupling+xyCoupling)
                if N2[i][j] != 0:
                    sum += J2*(zCoupling+xyCoupling)

        return sum

### Generate State Table (only for testing)

In [92]:
def generateStateTable(n,n0,N): # assumes ab > n0 > 0 and N = n choose n0
    states = []
    state = np.concatenate((np.ones(n-n0),-1*np.ones(n0))).astype(int)
    for i in range(0,N):
        states.append(np.copy(state))
        j = 0
        flag = True
        flip_count = 0
        up_count = 0
        while (flag): 
            if (j == n-1):
                for m in range(0,n):
                    if (state[m] != 1 and state[m+1] == 1):
                        flip_count += 1
                if (flip_count == 1):
                    break
            if (state[j] != 1):
                j += 1  
            elif (state[j+1] != 1):
                for m in range(0,j):
                    if (state[m] != 1 and state[m+1] == 1):
                        flip_count += 1
                    if (state[m] == 1):
                        up_count += 1
                if (flip_count == 1):
                    state[j],state[j+1] = state[j+1],state[j]
                    for k in range(1,up_count+1):
                        state[j-k],state[k-1] = state[k-1],state[j-k]
                else:
                    state[j],state[j+1] = state[j+1],state[j]
                flag = False
            else:
                j += 1
    return np.array(states)

### Test Energy Calculation

In [96]:
H = sparse.load_npz(PATH +'/4x4hamiltonian.npz')
H = H.toarray()

N = len(H)
randState = np.random.uniform(-1,1,N)
spikes = np.random.choice(range(0,N), size=20, replace=False)
for spike in spikes:
    randState[spike] *= np.random.normal(1000,100,1)[0]
randState /= np.linalg.norm(randState)

exactEnergy = randState @ H @ randState
print(exactEnergy)

spinors = np.array([[0,0]]*16,dtype=np.float64)
stateTable = generateStateTable(16,8,12870)
for basis in range(len(stateTable)):
    for i in range(16):
        if stateTable[basis][i] == 1:
            spinors[i][0] += randState[basis]
        else:
            spinors[i][1] += randState[basis]

stateObj = State(4,4,spinors)

wtfEnergy = stateObj.energy(1,0)
print(wtfEnergy)


-2.1645497723405382
-2.1645497723405382
34.562449598792654
